In [119]:
'''
Analysis on how the ownership of a token holder change
'''

'\nAnalysis on how the ownership of a token holder change\n'

In [120]:
from bs4 import BeautifulSoup
import urllib3
import re
import pandas
import numpy

In [130]:
baseUrl='https://etherscan.io/'
def html_convert_top100(tokenid, classname):
    '''
    Get the top 100 owners of the coin
    '''
    global baseUrl
    funUrl_chart='token/tokenholderchart/'
    pool=urllib3.PoolManager()
    r=pool.request('GET',baseUrl+funUrl_chart+tokenid)
    html=r.data
    soup = BeautifulSoup(html)
    table = soup.find("table", attrs=classname)
    headings = [th.get_text() for th in table.find("tr").find_all("th")]
    for idx in range(len(headings)):
        headings[idx]=str(headings[idx])
    datasets = []
    for row in table.find_all("tr")[1:]:
        dataset = (td.get_text() for td in row.find_all("td"))
        ls=list(dataset)
        datasets.append(ls)

    '''
    Clean the data
    '''
    for idx in range(len(datasets)):
        for idxx in range(len(datasets[idx])):
            tmp=re.sub(r'\([^)]*\)', '', datasets[idx][idxx])
            tmp=tmp.strip()
            tmp=str(tmp)
            datasets[idx][idxx]=tmp

    '''
    Create pandas dataframe and convert it to float
    '''
    df=pandas.DataFrame(datasets, columns=headings)
    df['Quantity (Token)']=df['Quantity (Token)'].astype(numpy.float64)
    df['Percentage']=df['Percentage'].str.strip('%')
    df['Percentage']=df['Percentage'].astype(numpy.float64)/100
    return df

def owners_t(ownerid, classname):
    transUrl='tokentxns?a='

def ICO_TOKEN(tokenid, tokenname):

    df=html_convert_top100(tokenid, "table table-hover ")
    '''
    Construct the ownership table
    '''
    owners={}
    for index, row in df.iterrows():
        owners[row['Address']]=row['Quantity (Token)']
    '''
    Construct the ownership transaction table
    '''
    owners_t={}
    for owner in owners:
        print owner
    
        
ICO_TOKEN('0x86fa049857e0209aa7d9e616f7eb3b3b78ecfdb0','EOS')

/usr/local/lib/python2.7/site-packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


0xdb7b85f792e7ff56abe78b03338ed24fbd088464
0xa3da370e7e81fd7889a0c3f050c46646a9d6a2ab
0x91136d2250fef45d08d3e34bf9e450a7067456d7
0x9c00c631b36ba5d93ed8f5279a4c55e6c2ba7c66
0xe3f5ed01e4990f990e1f0eb8d67c364c3c584ce6
0xa27b29df3664fe34a8d942477a8e1279ba0faca7
0x6dcd4cbcd6767d798a47a4bfcd0aaec3cba2a301
0xe7e561fc67d0008e590fd383fc0fe1571fb7b27e
0xff0cb0351a356ad16987e5809a8daaaf34f5adbe
0x1d73fe12a16f0299c1c9a01f324f7f37a34cca45
0xa6a1fd66f8eb61a4382c578150a48967966bb418
0xf6387de51ae0ea118c78a76f69966559d990b099
0x174443351e21d47ed9ab51517a301107d92ede64
0xed6fe5687f5b3b571a49398c797ffbeb8f306b76
0xf4b51b14b9ee30dc37ec970b50a486f37686e2a8
0x8a8f09adb437b2bfdbb4ae53ad82c7791ef443f1
0x1dba1131000664b884a1ba238464159892252d3a
0x505145a011b6f3c0b683d7e737f332226ddbc65a
0x4efd35e6668339e983134a1d214bdce0f785abb1
0xc0d57780bd9084f932a7ad9fc57d3aa18f1ab21c
0x06c234ddf496edd1b2257687a710c7ab5497b90b
0x9a7ea9286abb84b255bb5f8d74a0d81b3684ab99
0x42da8a05cb7ed9a43572b5ba1b8f82a0a6e263dc
0xd47297cdc